<a href="https://www.kaggle.com/code/kenny3s/enc-dec-times?scriptVersionId=170012838" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [11]:
dataset_dir = "/kaggle/input/vc-test/picture_materials"

In [12]:
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
from skimage import img_as_ubyte, io

# 获取dataset_dir目录下的所有文件名
image_files = os.listdir(dataset_dir)
# 转换为灰度图像
for image_file in image_files:
    image_path = os.path.join(dataset_dir, image_file)
    image = Image.open(image_path)
    gray_images = image.convert("L")
    #gray_images.save(os.path.join(dataset_dir, image_file))

In [13]:
# def block_no_expand(PIL_img):
#     image=np.array(PIL_img)
#     image_lab=cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
#     image_lab[:,:,-1]=np.interp(image_lab[:,:,-1],(0,255),(0,255/4))
#     image_rgb=cv2.cvtColor(image_lab,cv2.COLOR_HSV2RGB)
#     image_ht=Image.fromarray(image_rgb).convert("1")
#     image_ht=img_as_ubyte(image_ht)
#     return image_ht
def block_no_expand(PIL_img):
    image = np.array(PIL_img)
    # Scale the image intensity
    image = np.interp(image, (0, 255), (0, 255/4))
    # Convert the image to binary
    image_ht = Image.fromarray(image).convert("1")
    # Convert the binary image to 8-bit unsigned integer format
    image_ht = img_as_ubyte(image_ht)
    return image_ht
import math
import random

import cv2
from skimage.util import img_as_ubyte, view_as_blocks
from PIL import Image
import numpy as np
import os


def save_image(img_arr, output_path):
    """
    保存图片
    :param img_arr: Numpy数据类型
    :param output_path: 保存地址
    :return: None
    """
    output_dir = os.path.dirname(output_path)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    img = Image.fromarray(img_arr)
    img.save(output_path)


def img_bc(img_ht):  # 子块黑色像素计数
    """
    计算块中黑色像素数量
    :param img_ht:
    :return:
    """
    block = view_as_blocks(img_ht, (2, 2))
    black_count = []
    for (x, y) in np.ndindex(block.shape[:2]):
        b = block[x, y]
        black_sum = np.count_nonzero(b == 0)
        black_count.append(black_sum)
    ret = np.unique(black_count, return_counts=True)
    print(ret)
    return ret, img_ht


def error_diffusion(img, x, y, error=None):
    if not error:
        p = np.clip(img[x, y], 0, 255)
        img[x, y] = p // 128 * 255
        error = np.sum(p * 1.0 - img[x, y]) / np.size(img[x, y])
    s = np.array(img.shape[:2]) - 1
    if y < s[1]:
        img[x, y + 1] = np.clip(img[x, y + 1] + 7 / 16.0 * error, 0, 255)
    if x < s[0]:
        img[x + 1, y - 1] = np.clip(img[x + 1, y - 1] + 3 / 16.0 * error, 0, 255)
        img[x + 1, y] = np.clip(img[x + 1, y] + 5 / 16.0 * error, 0, 255)
    if x < s[0] and y < s[1]:
        img[x + 1, y + 1] = np.clip(img[x + 1, y + 1] + 1 / 16.0 * error, 0, 255)


def gray_level(img, levels=(2, 3)):
    img_bs = view_as_blocks(img, (2, 2))
    level_size = len(levels)
    block_size = 4
    for (x, y) in np.ndindex(img_bs.shape[:2]):
        error = np.sum(img_bs[x, y])
        mapped_level = math.floor(level_size / (block_size + 1) * (4 - error / 255))
        new_block = np.ones(block_size) * 255
        new_block[random.sample(range(block_size), levels[mapped_level])] = 0
        img_bs[x, y] = new_block.reshape(2, 2)
        error = (error - np.sum(new_block)) / block_size
        error_diffusion(img_bs, x, y, error)


def adjust_brightness(PIL_img, n=4, correct=False):
    """
    修改图片亮度
    :param path: 图片路径
    :param n: 暗度调节
    :param correct: 是否色块修正
    :return: 加密图片（numpu）
    """
    
    image = np.array(PIL_img)
    # Scale the image intensity
    image = np.interp(image, (0, 255), (0, 255/4))
    # Convert the image to binary
    image_ht = Image.fromarray(image).convert("1")
    # Convert the binary image to 8-bit unsigned integer format
    image_ht = img_as_ubyte(image_ht)
    if correct:
        gray_level(image_ht, (3, 4))
    return image_ht


def random_index(my_list, n):
    """
    在列表中随机抽取元素
    :param my_list: 被抽取的列表
    :param n: 抽取个数
    :return: 抽取的列表和未被抽取的列表
    """
    selected_elements = random.sample(my_list, n)  # 被抽取的列表
    remaining_elements = [element for element in my_list if element not in selected_elements]  # 未被抽取的列表
    return selected_elements, remaining_elements


def blocks_2x2(img_en_block, share1_point, share2_point):
    block1 = np.ones((2, 2), dtype=np.uint8) * 255  # 全白2x2的单位块
    block2 = np.copy(block1)
    block_index = np.argwhere(img_en_block == 0)  # 获取img_en_block值为0的索引
    block_index_list = [(row, col) for row, col in block_index]  # 将索引存储到列表中
    if len(block_index_list) == 4 and share1_point == 0 and share2_point == 0:  # 原图是黑色，分图1是黑色，分图2是黑色
        block1_index, one = random_index(block_index_list, 3)
        block2_index, other = random_index(block1_index, 2)
        block2_index = block2_index + one
        for index in block1_index:
            block1[index] = 0
        for index in block2_index:
            block2[index] = 0
    elif len(block_index_list) == 4 and share1_point == 0 and share2_point == 255:  # 原图是黑色，分图1是黑色，分图2是白色
        block1_index, one = random_index(block_index_list, 3)
        block2_index, other = random_index(block1_index, 1)
        block2_index = block2_index + one
        for index in block1_index:
            block1[index] = 0
        for index in block2_index:
            block2[index] = 0
    elif len(block_index_list) == 4 and share1_point == 255 and share2_point == 0:  # 原图是黑色，分图1是白色，分图2是黑色
        block2_index, one = random_index(block_index_list, 3)
        block1_index, other = random_index(block2_index, 1)
        block1_index = block1_index + one
        for index in block1_index:
            block1[index] = 0
        for index in block2_index:
            block2[index] = 0
    elif len(block_index_list) == 4 and share1_point == 255 and share2_point == 255:  # 原图是黑色，分图1是白色，分图2是白色
        block1_index, block2_index = random_index(block_index_list, 2)
        for index in block1_index:
            block1[index] = 0
        for index in block2_index:
            block2[index] = 0
    elif len(block_index_list) == 3 and share1_point == 0 and share2_point == 0:  # 原图是白色，分图1是黑色，分图2是黑色
        block1 = img_en_block
        block2 = img_en_block
    elif len(block_index_list) == 3 and share1_point == 0 and share2_point == 255:  # 原图是白色，分图1是黑色，分图2是白色
        block1 = img_en_block
        block2_index, other = random_index(block_index_list, 2)
        for index in block2_index:
            block2[index] = 0
    elif len(block_index_list) == 3 and share1_point == 255 and share2_point == 0:  # 原图是白色，分图1是白色，分图2是黑色
        block2 = img_en_block
        block1_index, other = random_index(block_index_list, 2)
        for index in block1_index:
            block1[index] = 0
    elif len(block_index_list) == 3 and share1_point == 255 and share2_point == 255:  # 原图是白色，分图1是白色，分图2是黑色
        block1_index, one = random_index(block_index_list, 2)
        block2_index, other = random_index(block1_index, 1)
        block2_index = block2_index + one
        for index in block1_index:
            block1[index] = 0
        for index in block2_index:
            block2[index] = 0
    else:
        print("ERROR")
    return block1, block2


def no_pixels_expand_meaning_share(img_en, colored_share1_path, colored_share2_path):
    """
    无像素拓展有意义分享方案
    :param img_en: 加密图像(半色调，numpy)
    :param colored_share1: 彩色图像分享1
    :param colored_share2: 彩色图像分享2
    :return: 分享1，分享2
    """
    colored_share1 = Image.open(colored_share1_path).convert('L').convert('RGB')
    colored_share2 = Image.open(colored_share2_path).convert('L').convert('RGB')
    # 修改colored_share1和colored_share2尺寸为img_en的1/4
    colored_share1 = colored_share1.resize(tuple(int(element / 2) for element in img_en.shape[:2]))
    colored_share2 = colored_share2.resize(tuple(int(element / 2) for element in img_en.shape[:2]))
    # 将分享转换成半色调图片
    share1_ht = np.array(colored_share1.convert("1"),dtype=np.uint8)*255
    share2_ht = np.array(colored_share2.convert("1"),dtype=np.uint8)*255
    # 将加密图像分块
    img_en_block = view_as_blocks(img_en, (2, 2))
    # 创建两个等大的空分享
    share1 = np.ones_like(img_en) * 255
    share2 = np.ones_like(img_en) * 255
    share1_block = view_as_blocks(share1, (2, 2))
    share2_block = view_as_blocks(share2, (2, 2))
    # 加密
    start = time.perf_counter()
    for i in range(img_en_block.shape[0]):
        for j in range(img_en_block.shape[1]):
            share1_block[i, j], share2_block[i, j] = blocks_2x2(img_en_block[i,j], share1_ht[i, j], share2_ht[i, j])
    enc_time = time.perf_counter() - start
    return share1, share2, enc_time


def Test():
    img_ab = adjust_brightness('../input_image/picture_materials/Airplane.png', correct=True)
    img_bc(img_ab)
    save_image(img_ab, '../output_image/no_pixels_expand_meaning_share/img_ab.png')
    share1, share2 = no_pixels_expand_meaning_share(img_ab, '../input_image/picture_materials/Baboon.png',
                                                    '../input_image/picture_materials/Barbara.png')
    save_image(share1, '../output_image/no_pixels_expand_meaning_share/share1.png')
    save_image(share2, '../output_image/no_pixels_expand_meaning_share/share2.png')
    save_image(share1 & share2, '../output_image/no_pixels_expand_meaning_share/share1&share2.png')
    print(share1.shape)
    print(share2.shape)
    print((share1&share2).shape)

def block_test():
    block  = np.array([[0, 255], [0,0]])
    share1 = 255
    share2 = 0
    share1_block, share2_block = blocks_2x2(block,share1,share2)
    print(f'加密图像块：{block}')
    print(f'分享1块：{share1_block}')
    print(f'分享2块：{share2_block}')

In [14]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP, DES3, AES
from Crypto.Util.Padding import pad, unpad
from Crypto.Random import get_random_bytes
import time

# RSA加密和解密
def rsa_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()

    # 生成RSA密钥
    key = RSA.generate(2048)
    cipher = PKCS1_OAEP.new(key)

    # 计算最大明文长度
    max_length = 128

    # 将图像分割成小块
    chunks = [image[i:i+max_length] for i in range(0, len(image), max_length)]

    # 加密
    encrypted_chunks = [cipher.encrypt(chunk) for chunk in chunks]
    enc_time = time.perf_counter() - start

    # 解密
    start = time.perf_counter()
    decrypted_chunks = [cipher.decrypt(chunk) for chunk in encrypted_chunks]
    dec_time = time.perf_counter() - start

    # 将解密后的小块重新组合成完整的图像
    decrypted = b''.join(decrypted_chunks)

    return enc_time, dec_time

def des3_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()


    # 生成3DES密钥和初始化向量
    key = get_random_bytes(24)
    iv = get_random_bytes(8)

    # 创建一个新的DES3对象，使用CBC模式
    cipher_enc = DES3.new(key, DES3.MODE_CBC, iv)

    # 加密
    encrypted = cipher_enc.encrypt(pad(image, DES3.block_size))
    enc_time = time.perf_counter() - start

    # 创建一个新的DES3对象，使用CBC模式，用于解密
    cipher_dec = DES3.new(key, DES3.MODE_CBC, iv)

    # 解密
    start = time.perf_counter()
    decrypted = unpad(cipher_dec.decrypt(encrypted), DES3.block_size)
    dec_time = time.perf_counter() - start

    return enc_time, dec_time

# AES加密和解密
def aes_enc_dec(image):
    # 将图像转换为字节流
    start = time.perf_counter()

    # 生成AES密钥和初始化向量
    key = get_random_bytes(32)
    iv = get_random_bytes(16)

    # 创建一个新的AES对象，使用CBC模式
    cipher_enc = AES.new(key, AES.MODE_CBC, iv)

    # 加密
    start = time.perf_counter()
    encrypted = cipher_enc.encrypt(pad(image, AES.block_size))
    enc_time = time.perf_counter() - start

    # 创建一个新的AES对象，使用CBC模式，用于解密
    cipher_dec = AES.new(key, AES.MODE_CBC, iv)

    # 解密
    start = time.perf_counter()
    decrypted = unpad(cipher_dec.decrypt(encrypted), AES.block_size)
    dec_time = time.perf_counter() - start

    return enc_time, dec_time

def no_expand_enc_dec(img,enc_path1,enc_path2):
    # 加密
    #img = img.convert('RGB')
    img_ab = adjust_brightness(img, correct=True)
    #img_bc(img_ab)
    share1, share2, enc_time = no_pixels_expand_meaning_share(img_ab, enc_path1, enc_path2)
    
    # 解密
    start = time.perf_counter()
    dec = share1&share2
    dec_time = time.perf_counter() - start
    
    Image.fromarray(dec).save('encrypted.png')
    print(f'Encrypted image size: {os.path.getsize("encrypted.png") / 1024:.2f} KB')

    return enc_time, dec_time

# 对每张图像进行加密和解密，并计算耗时
for image_file in image_files:
    ori_size = os.path.getsize(os.path.join(dataset_dir, image_file))
    # 打开图像文件，并转换为灰度图像
    img_gray = Image.open(os.path.join(dataset_dir, image_file)).convert('L')
    img = img_gray.tobytes()

    img_rgb = Image.open(os.path.join(dataset_dir, image_file))
    # 计算RSA加密和解密的耗时
    rsa_enc_time, rsa_dec_time = rsa_enc_dec(img)
    rsa_enc_time, rsa_dec_time = (round(rsa_enc_time * 1000, 2), round(rsa_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算DES加密和解密的耗时
    des3_enc_time, des3_dec_time = des3_enc_dec(img)
    des3_enc_time, des3_dec_time = (round(des3_enc_time * 1000, 2), round(des3_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算AES加密和解密的耗时
    aes_enc_time, aes_dec_time = aes_enc_dec(img)
    aes_enc_time, aes_dec_time = (round(aes_enc_time * 1000, 2), round(aes_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    # 计算不扩展加密和解密的耗时
    no_expand_enc_time, no_expand_dec_time = no_expand_enc_dec(img_gray, f"{dataset_dir}/Lena.png", f"{dataset_dir}/Peppers.png")
    no_expand_enc_time, no_expand_dec_time = (round(no_expand_enc_time * 1000, 2), round(no_expand_dec_time * 1000, 2))  # 转换为毫秒并保留两位小数

    rsa_total_time = rsa_enc_time + rsa_dec_time
    des3_total_time = des3_enc_time + des3_dec_time
    aes_total_time = aes_enc_time + aes_dec_time
    no_expand_total_time = no_expand_enc_time + no_expand_dec_time

    print(f'Image: {image_file}, Size: {ori_size / 1024:.2f} KB')
    print(f'RSA: Enc {rsa_enc_time} ms, Dec {rsa_dec_time} ms, Total {rsa_total_time} ms')
    print(f'3DES: Enc {des3_enc_time} ms, Dec {des3_dec_time} ms, Total {des3_total_time} ms')
    print(f'AES: Enc {aes_enc_time} ms, Dec {aes_dec_time} ms, Total {aes_total_time} ms')
    print(f'NE_VC: Enc {no_expand_enc_time} ms, Dec {no_expand_dec_time} ms, Total {no_expand_total_time} ms')

Encrypted image size: 5.76 KB
Image: Baboon.png, Size: 159.17 KB
RSA: Enc 1157.23 ms, Dec 1517.54 ms, Total 2674.77 ms
3DES: Enc 3.49 ms, Dec 3.18 ms, Total 6.67 ms
AES: Enc 0.2 ms, Dec 0.21 ms, Total 0.41000000000000003 ms
NE_VC: Enc 1159.31 ms, Dec 0.02 ms, Total 1159.33 ms
Encrypted image size: 5.51 KB
Image: Butterfly.png, Size: 128.67 KB
RSA: Enc 915.66 ms, Dec 1287.42 ms, Total 2203.08 ms
3DES: Enc 3.19 ms, Dec 2.91 ms, Total 6.1 ms
AES: Enc 0.18 ms, Dec 0.19 ms, Total 0.37 ms
NE_VC: Enc 1120.86 ms, Dec 0.02 ms, Total 1120.8799999999999 ms
Encrypted image size: 5.97 KB
Image: House.png, Size: 110.08 KB
RSA: Enc 2534.45 ms, Dec 1268.48 ms, Total 3802.93 ms
3DES: Enc 3.21 ms, Dec 2.94 ms, Total 6.15 ms
AES: Enc 0.19 ms, Dec 0.2 ms, Total 0.39 ms
NE_VC: Enc 1081.11 ms, Dec 0.02 ms, Total 1081.1299999999999 ms
Encrypted image size: 5.50 KB
Image: Peppers.png, Size: 120.83 KB
RSA: Enc 1871.0 ms, Dec 1281.77 ms, Total 3152.77 ms
3DES: Enc 3.27 ms, Dec 2.96 ms, Total 6.23 ms
AES: Enc 0.